In [ ]:
pip install beautifulsoup4 

In [ ]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import datetime
import dateutil.relativedelta
import pandas as pd
import re

In [ ]:
url = 'https://cbr.ru/currency_base/dynamics/'
page = requests.get(url)

In [ ]:
print(page.status_code) #проверяем соединение

200


In [ ]:
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
today = datetime.datetime.now()
delta1 = datetime.timedelta(hours=3, minutes=0)
today=today + delta1
datenow=today.strftime("%d.%m.%Y")

delta2 = dateutil.relativedelta.relativedelta(years=1)
notoday=today-delta2
datepast=notoday.strftime("%d.%m.%Y")

datenowkek=today.strftime("%m%%2F%d%%2F%Y")
datepastkek=notoday.strftime("%m%%2F%d%%2F%Y")

In [ ]:
class_input_label = soup.find("label", class_="input_label")
labels=[]
nums=[]
kr=[]
for i in range(len(class_input_label.findAll('option'))):
  find_options=class_input_label.findAll('option')[i]
  con_with_del=find_options.contents[0]
  num=find_options.get_attribute_list('value')[0]
  nums.append(num)
  label=' '.join(con_with_del.split())
  labels.append(label)


  #url2=f'https://cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={num}&UniDbQuery.From={datepast}&UniDbQuery.To={datenow}'
  #url2='https://cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={0}&UniDbQuery.From={1}&UniDbQuery.To={2}'.format(num,datepast,datenow)
  url2='https://cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={0}&UniDbQuery.From={1}&UniDbQuery.To={2}'.format(num,'01.01.1992',datenow)
  url3=url2
  page2 = requests.get(url3)
  soup2 = BeautifulSoup(page2.text, "html.parser")
  k=soup2.find("div", class_="table-caption")
  if k==None:
    kr.append('')
  else:
    textik=k.get_text()
    m = re.findall(r"\(([A-Za-z]+).+\)",textik)
    if len(m)==0:
      kr.append('')
    else:
      kr.append(m.pop())
  


In [ ]:
conn = sqlite3.connect("drive/My Drive/mydata.db")
cur=conn.cursor()

In [ ]:
cur.execute("""CREATE TABLE IF NOT EXISTS names(
   id INT PRIMARY KEY,
   bname TEXT,
   sname TEXT);
""")
conn.commit()

In [ ]:
id=range(1,len(kr))
c = [(x,y,z) for x,y,z in zip(id,labels,kr)]

In [ ]:
cur.executemany("INSERT INTO names VALUES(?, ?, ?);", c)
conn.commit()

In [ ]:
np2=[]
while len(np2)<3:
  print('Введите полное наименование валюты')
  k=input()
  if k not in labels:
    print('Ошибка ввода. Такой валюты не существует.')
  else: 
    np2.append(k)

Введите полное наименование валюты
Австралийский доллар
Введите полное наименование валюты
Белорусский рубль
Введите полное наименование валюты
Канадский доллар


In [ ]:
mass=[]
for n in np2:
  aaa=nums[labels.index(n)]
 # url2='https://cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={0}&UniDbQuery.From={1}&UniDbQuery.To={2}'.format(a,datepast,datenow)
 # 'https://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True&mode=1&VAL_NM_RQ={a}&From={datepast}&To={datenow}&FromDate=10%2F14%2F2020&ToDate=10%2F21%2F2020'.format(a,datepast,datenow)
  url2=f'https://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True&mode=1&VAL_NM_RQ={aaa}&From={datepast}&To={datenow}&FromDate={datepastkek}&ToDate={datenowkek}'#.format(aaa,datepast,datenow)
  excel = pd.read_excel(url2)
  mass.append(excel)

In [ ]:
cur.execute("""CREATE TABLE IF NOT EXISTS rates1(
   data TEXT PRIMARY KEY,
   curs TEXT);
   """)
conn.commit()

m0=mass[0]

r1 = [(x,y) for x,y in zip(list(m0['data'].astype(str)),list(m0['curs'].astype(str)))]
head1 = ('data', nums[labels.index(np2[0])])
cur.execute("INSERT INTO rates1 VALUES(?, ?);", head1)
conn.commit()
cur.executemany("INSERT INTO rates1 VALUES(?, ?);", r1)
conn.commit()

In [ ]:
cur.execute("""CREATE TABLE IF NOT EXISTS rates2(
   data TEXT PRIMARY KEY,
   curs TEXT);
   """)
conn.commit()

m1=mass[1]

r2 = [(x,y) for x,y in zip(list(m1['data'].astype(str)),list(m1['curs'].astype(str)))]
head2 = ('data', nums[labels.index(np2[1])])
cur.execute("INSERT INTO rates2 VALUES(?, ?);", head2)
conn.commit()
cur.executemany("INSERT INTO rates2 VALUES(?, ?);", r2)
conn.commit()

In [ ]:
cur.execute("""CREATE TABLE IF NOT EXISTS rates3(
   data TEXT PRIMARY KEY,
   curs TEXT);
   """)
conn.commit()

m2=mass[2]

r3 = [(x,y) for x,y in zip(list(m2['data'].astype(str)),list(m2['curs'].astype(str)))]
head3 = ('data', nums[labels.index(np2[2])])
cur.execute("INSERT INTO rates3 VALUES(?, ?);", head3)
conn.commit()
cur.executemany("INSERT INTO rates3 VALUES(?, ?);", r3)
conn.commit()

In [ ]:
conn.close()